In [1]:
!pip install owlready2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.3/27.3 MB 56.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for owlready2: filename=owlready2-0.48-cp311-cp311-linux_x86_64.whl size=24551720 sha256=42d07c5931989cb207f5d977c19b19d00671dd79e2d04a8a3e1945cf581cb7c6
  Stored in directory: /root/.cache/pip/wheels/2a/4f/b2/88d834aab03077e1611b46825f45c06ac4db07b77ee45eadd5
Successfully built owlready2


In [ ]:
from owlready2 import get_ontology, ThingClass, Restriction, SOME

# Load directly from the official Pizza Ontology URL
url = "https://protege.stanford.edu/ontologies/pizza/pizza.owl"
onto = get_ontology(url).load(format="rdfxml")

el_axioms = []

for cls in onto.classes():
    for ax in cls.is_a:
        if isinstance(ax, ThingClass) and isinstance(cls, ThingClass):
            el_axioms.append(f"{cls.name} ⊑ {ax.name}")
        elif isinstance(ax, Restriction) and ax.type == SOME and isinstance(ax.value, ThingClass):
            el_axioms.append(f"{cls.name} ⊑ ∃{ax.property.name}.{ax.value.name}")

# Show a sample
for ax in el_axioms[:10]:
    print(ax)

Pizza ⊑ Food
Pizza ⊑ ∃hasBase.PizzaBase
PizzaBase ⊑ Food
Food ⊑ DomainConcept
Spiciness ⊑ ValuePartition
PizzaTopping ⊑ Food
American ⊑ NamedPizza
American ⊑ ∃hasTopping.MozzarellaTopping
American ⊑ ∃hasTopping.PeperoniSausageTopping
American ⊑ ∃hasTopping.TomatoTopping


In [ ]:
from pathlib import Path
from google.colab import files

Path("pizza_el_axioms.txt").write_text("\n".join(sorted(set(el_axioms))))
files.download("pizza_el_axioms.txt")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [8]:
import types
from owlready2 import get_ontology, Thing, Restriction, SOME, onto_path, ObjectProperty

# Load Pizza OWL DL ontology
pizza = get_ontology("https://protege.stanford.edu/ontologies/pizza/pizza.owl").load()

# Create EL-only ontology
onto_path.append(".")
el_onto = get_ontology("http://example.org/pizza-el.owl")

with el_onto:
    class_map = {}
    prop_map = {}

    # Create clean class entities
    for cls in pizza.classes():
        class_map[cls.name] = types.new_class(cls.name, (Thing,))

    # Create clean object properties
    for prop in pizza.object_properties():
        prop_map[prop.name] = types.new_class(prop.name, (ObjectProperty,))

    # Transfer EL-safe axioms
    for cls in pizza.classes():
        new_cls = class_map.get(cls.name)
        if not new_cls:
            continue

        for ax in cls.is_a:
            if isinstance(ax, Thing) and ax.name in class_map:
                new_cls.is_a.append(class_map[ax.name])

            elif isinstance(ax, Restriction) and ax.type == SOME:
                prop = prop_map.get(ax.property.name)
                filler = class_map.get(ax.value.name)
                if prop and filler:
                    new_cls.is_a.append(prop.some(filler))

# Save
el_onto.save(file="pizza-el.owl", format="rdfxml")
print("Saved pizza-el.owl (EL-only axioms)")

✅ Saved pizza-el.owl (EL-only axioms)
